In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
%matplotlib inline
import numpy as np
import numpy.random as nd
import pandas as pd
import math
import matplotlib.pyplot as plt

import os
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from IPython.display import display

import random

In [3]:
pl = pd.read_csv('Premier League 17-18 abbrev.csv')
liga = pd.read_csv('LaLiga1718_abbrev.csv')
ligue = pd.read_csv('FranceLigue1_season1718_abbrev.csv')
games = pd.read_csv('soccer_data_abbrev.csv', encoding = 'ISO-8859-1')

In [4]:
def abbv_team (colu):
    lst = colu.tolist()
    ret = []
    for team in lst:
        ret.append(team[:4])
    return np.array(ret)

In [5]:
abbvteam = abbv_team(games['home'])
games['game'] = abbvteam+games['date']
games['game'].astype(str)

0       Arse11/08/17
1       Watf12/08/17
2       Chel12/08/17
3       Crys12/08/17
4       Ever12/08/17
5       Sout12/08/17
6       West12/08/17
7       Brig12/08/17
8       Newc13/08/17
9       Manc13/08/17
10      Swan19/08/17
11      Bour19/08/17
12      Burn19/08/17
13      Leic19/08/17
14      Live19/08/17
15      Sout19/08/17
16      Stok19/08/17
17      Hudd20/08/17
18      Tott20/08/17
19      City21/08/17
20      Bour26/08/17
21      Crys26/08/17
22      Hudd26/08/17
23      Newc26/08/17
24      Watf26/08/17
25      Manc26/08/17
26      Chel27/08/17
27      West27/08/17
28      Live27/08/17
29      Tott27/08/17
            ...     
1100    Athl05/05/18
1101    Celt05/05/18
1102    Vill05/05/18
1103    Mala06/05/18
1104    Atle06/05/18
1105    Las 06/05/18
1106    Barc06/05/18
1107    Lega07/05/18
1108    Barc09/05/18
1109    Sevi09/05/18
1110    RSoc12/05/18
1111    DAla12/05/18
1112    Depo12/05/18
1113    Eiba12/05/18
1114    Geta12/05/18
1115    Giro12/05/18
1116    Real1

In [6]:
plabbvteam = abbv_team(pl['HomeTeam'])
pl['game'] = plabbvteam+pl['Date']
pl['game'].astype(str)

0      Arse11/08/17
1      Brig12/08/17
2      Chel12/08/17
3      Crys12/08/17
4      Ever12/08/17
5      Sout12/08/17
6      Watf12/08/17
7      West12/08/17
8      Manc13/08/17
9      Newc13/08/17
10     Bour19/08/17
11     Burn19/08/17
12     Leic19/08/17
13     Live19/08/17
14     Sout19/08/17
15     Stok19/08/17
16     Swan19/08/17
17     Hudd20/08/17
18     Tott20/08/17
19     City21/08/17
20     Bour26/08/17
21     Crys26/08/17
22     Hudd26/08/17
23     Manc26/08/17
24     Newc26/08/17
25     Watf26/08/17
26     Chel27/08/17
27     Live27/08/17
28     Tott27/08/17
29     West27/08/17
           ...     
350    Swan28/04/18
351    Manc29/04/18
352    West29/04/18
353    Tott30/04/18
354    Brig04/05/18
355    Bour05/05/18
356    Ever05/05/18
357    Leic05/05/18
358    Stok05/05/18
359    Watf05/05/18
360    West05/05/18
361    Arse06/05/18
362    Chel06/05/18
363    City06/05/18
364    Swan08/05/18
365    Chel09/05/18
366    Leic09/05/18
367    City09/05/18
368    Tott09/05/18


In [7]:
ligaabbvteam = abbv_team(liga['HomeTeam'])
liga['game'] = ligaabbvteam + liga['Date']
liga['game'].astype(str)

0      Lega18/08/17
1      Vale18/08/17
2      Celt19/08/17
3      Giro19/08/17
4      Sevi19/08/17
5      Athl20/08/17
6      Barc20/08/17
7      Depo20/08/17
8      Leva21/08/17
9      Mala21/08/17
10     Real25/08/17
11     RSoc25/08/17
12     DAla26/08/17
13     Giro26/08/17
14     Las 26/08/17
15     Leva26/08/17
16     Eiba27/08/17
17     Espa27/08/17
18     Geta27/08/17
19     RMad27/08/17
20     Lega08/09/17
21     Barc09/09/17
22     RMad09/09/17
23     Sevi09/09/17
24     Vale09/09/17
25     Athl10/09/17
26     Celt10/09/17
27     Depo10/09/17
28     Vill10/09/17
29     Mala11/09/17
           ...     
350    Celt05/05/18
351    Giro05/05/18
352    Vill05/05/18
353    Atle06/05/18
354    Barc06/05/18
355    Las 06/05/18
356    Mala06/05/18
357    Lega07/05/18
358    Barc09/05/18
359    Sevi09/05/18
360    DAla12/05/18
361    Real12/05/18
362    Eiba12/05/18
363    Geta12/05/18
364    Giro12/05/18
365    Depo12/05/18
366    RMad12/05/18
367    RSoc12/05/18
368    Espa13/05/18


In [8]:
ligueabbvteam = abbv_team(ligue['HomeTeam'])
ligue['game'] = ligueabbvteam + ligue['Date']
ligue['game'].astype(str)

0      AS M04/08/17
1      Olym05/08/17
2      FC M05/08/17
3      Mont05/08/17
4      Pari05/08/17
5      AS S05/08/17
6      ES T05/08/17
7      Ange06/08/17
8      Lill06/08/17
9      OdMe06/08/17
10     OGC 11/08/17
11     Stad11/08/17
12     Amie12/08/17
13     FC G12/08/17
14     Stad12/08/17
15     FC N12/08/17
16     Toul12/08/17
17     Dijo13/08/17
18     EA G13/08/17
19     RC S13/08/17
20     FC M18/08/17
21     Olym19/08/17
22     Mont19/08/17
23     OGC 19/08/17
24     Stad19/08/17
25     AS S19/08/17
26     ES T19/08/17
27     Lill20/08/17
28     OdMe20/08/17
29     Pari20/08/17
           ...     
350    Amie04/05/18
351    Stad06/05/18
352    Dijo06/05/18
353    Olym06/05/18
354    OdMe06/05/18
355    FC M06/05/18
356    FC N06/05/18
357    Stad06/05/18
358    AS S06/05/18
359    Toul06/05/18
360    EA G11/05/18
361    Amie12/05/18
362    Ange12/05/18
363    FC G12/05/18
364    Lill12/05/18
365    AS M12/05/18
366    Mont12/05/18
367    OGC 12/05/18
368    Pari12/05/18


In [16]:
combined_leagues = pd.concat([pl, ligue, liga])

/Users/marcusheijer/new_p3_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
merged_games = games.merge(combined_leagues, on = 'game', how = 'inner')

In [28]:
merged_games.to_csv('merged_games.csv')

In [44]:
marketvalue = pd.read_csv('marketvalue_abbrev.csv')
marketvalue_home = marketvalue[['HomeTeam', 'total market value', 'age']]
marketvalue_away = marketvalue[['AwayTeam', 'total market value', 'age']]

In [42]:
merge_home_games = merged_games.merge(marketvalue_home, on = 'HomeTeam', how = 'inner')

In [43]:
merge_home_games.to_csv('merge_home_team.csv')

In [45]:
merge_home_games.rename(columns = {'total market value': 'home market value', 'age': 'home age'}, inplace = True)

In [47]:
merge_all_games = merge_home_games.merge(marketvalue_away, on = 'AwayTeam', how = 'inner')
merge_all_games.rename(columns = {'total market value': 'away market value', 'age': 'away age'}, inplace = True)

In [48]:
merge_all_games.to_csv('merged_marketvalue.csv')